<a href="https://colab.research.google.com/github/drscook/MathVGerrmandering_CMAT_2022/blob/main/Part1_MAA_TX22_Redistricting_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mathematical Association of America - Texas Section
# 2022 Annual Meeting
# Workshop on Mathematical Tools to Fight Gerrymandering
# Part I - Ensemble Analysis with GerryChain



---

## Event Information

### Time, Date, & Location
*7:00pm - 9:00pm*  
*March 31*  
*Gateway 42*  
*UNT Gateway Center*  
*Denton, TX*  


**Link to this Jupyter Notebook: https://tinyurl.com/gerrychain** 

Zoom link: https://unt.zoom.us/j/83519091082

GitHub Repo: https://github.com/drscook/Redistricting_Workshop_MAA_TX_2022

Video: https://youtu.be/a0zgVTTcFiQ

### Contributors

Workshop Facilitators:
- Dr. Scott Cook<sup>4</sup>
- Jaryd Domine<sup>3</sup>
- Cody Drolet<sup>4</sup>
- Dr. Will Hager<sup>5</sup>
- Mason McCallum<sup>3</sup>
- Dr. Betseygail Rand<sup>5</sup>
- Vianey Rangel<sup>4</sup>

Part I notebook contributors:
- Dr. Scott Cook<sup>4</sup>
- Diana Dinh-Andrus<sup>3</sup>
- Dr. Will Hager<sup>5</sup>
- Anthony Pizzimenti<sup>2</sup>
- Casey Sutton<sup>4</sup>
- Maria Tovar<sup>4</sup>
- Preston Ward<sup>4</sup>

Part II notebook contributors:
- Dr. Scott Cook<sup>4</sup>

Special Thanks:
- Metric Geometry & Gerrymandering Group
    - Dr. Moon Duchin<sup>6</sup>
- Math For Unbiased Maps Texas
    - Dr. Andrea Barreiro<sup>3</sup>
    - Dr. Matt Lockard<sup>3</sup>
    - Dr. Scott Norris<sup>3</sup>
    - Robert Meyers
    - Dr. Dustin Potter<sup>1</sup>
    - Dr. Brandi Stigler<sup>3</sup>


<sup>1</sup>Collin College, <sup>2</sup>Iowa State Univ, <sup>3</sup>Southern Methodist Univ, <sup>4</sup>Tarleton State Univ, <sup>5</sup>Texas Lutheran State Univ, <sup>6</sup>Tufts Univ

### Workshop Agenda

Ensemble analysis of electorial districting plans consists of several steps:
1. Get data - Retrieve, prepare, and combine geographic, demographic, and electoral data and assemble into a single dataset
2. Generate ensemble - Feed this dataset into tools like GerryChain to generate an ensemble of districting plans
3. Analyze - Statistically compare a proposed/enacted plan against the ensemble

Part I of this workshop focuses on steps 2 & 3 and assumes step 1 has already been completed.  Steps 2 & 3 are more interesting and satisfying mathematically and politically.  Step 1 is more tedious and requires more complex code, but is essential if you aim to do something beyond the scope of the dataset we provided.

Part II focuses on step 1 - retrieving, preparing, and combining data.  We will probably not discuss every line of code, but we'll hit as many highlights as time allows.

### Applications

 
  - Google Colab - What you are seeing right now.  Free, cloud based Python distribution from Google.  Like Google docs for Python.  Side note - allows you to attach GPU or TPU devices too for parallel computing (though we won't use that in this workshop)
  - Pandas - Data analysis package for Python.  We make use of its dataframes in this talk.
    - [Pandas Documentation](https://pandas.pydata.org/docs/)
  - GeoPandas - A Python package that allows the addition of geographic information to Pandas dataframes.
    - [Geopandas Documentation](https://geopandas.org/en/stable/docs.html)
  - Networkx - A Python package that creates, edits, and analyzes graphs.
    - [Networkx Documentation](https://networkx.github.io/)  
  - GerryChain - Python package to implement mathematical tools for gerrymandering.  Specifically designed to run Markov Chain Monte Carlo (MCMC) methods with a wide range of metrics.  Developed during the summer 2018 [Voting Rights Data Institute](https://sites.tufts.edu/vrdi/).
    - [GerryChain Documentation](https://gerrychain.readthedocs.io/en/latest/)
  -GerryChainJulia - Gerrychain in a Julia package.  The Julia implementation of Gerrychain tends to run faster than the Python implementation.
    - [GerryChainJulia Documentation](https://mggg.github.io/GerryChainJulia/stable/) 
  


---
## 0 - Before You Start

**Make a personal copy of this Colab notebook in your Google Drive so you can add, edit, take notes, comment, execute code, etc.  File → Save a copy in Drive**
 

---
## 1 - Install Gerrychain

In the following section we install the GerryChain library and import the functions we will need to run MCMC methods.  This package will create a collection of partitions against which we can compare real world partitions.

In [ ]:
#  This takes 3 minutes. You will receive an error message "Your session crashed 
#  for an unknown reason."  This happens because condacolab restarts the kernel 
#  of this worksheet.   

# installs condacolab, which allows installation of conda
! pip install -q condacolab  

# installs conda, which allows installation of mamba
import condacolab
condacolab.install()

In [ ]:
# try this first - its much faster if it works
! mamba install -q -y -c conda-forge gerrychain geopandas

# if that fails, use the slower
# ! conda install -q -y -c conda-forge gerrychain geopandas

from IPython import get_ipython
get_ipython().kernel.do_shutdown(True)

---
## 2 - Create GeoDataFrame from Imported Data

### a - Download a Data File and Load It Into a GeoDataFrame

The GeoJSON file downloaded in the cell below can also be found at the following link:  [Texas VTD Dataset](https://drive.google.com/file/d/1i-wOVyGf2RnQJSpCxDKlVBoJjVm-Pskq/view?usp=sharing).

In [ ]:
#  This takes 2 minutes.

#  Installs a downloader for Google Drive files.
!pip install gdown 

#  Downloads a .zip file from Google Drive.  Saves to Colab's local disk.  This 
#  will be lost when the runtime is restarted.  If you wish to keep it, download 
#  from the 'Files' tab on the left edge of the screen.
!gdown "1i-wOVyGf2RnQJSpCxDKlVBoJjVm-Pskq" -O "VTD-Data.zip"  #

#  Unzips the downloaded file.
!unzip "/content/VTD-Data.zip"

In [ ]:
#  GeoPandas allows us to work with dataframes containing geometric data.
import geopandas as gpd

#  Creates a geodataframe from the unzipped file.
gdf = gpd.read_file('/content/VTD_Data (1).geojson')

Note that GeoPandas can create GeoDataFrames from a variety of different filetypes.  See the [documentation](https://geopandas.org/en/stable/docs/user_guide/io.html) for more details.

### b - Check the GeoDataFrame

Let's take a look at it in table form.

In [ ]:
gdf

Let's plot the GeoDataFrame's geometries.  You should see Texas after running the cell.

In [ ]:
gdf.plot(column="county", figsize = (8,8), cmap = 'tab20')

---
## Optional - Using GeoDataFrames

There are many ways to read and view the data in our GeoDataFrame.

### Quick Overviews.
We can get a quick look at the top or bottom of the table.

In [ ]:
gdf.head(3)

In [ ]:
gdf.tail(4)

### Calling Entries from DataFrames.

We can select an entry using its index and column name...

In [ ]:
gdf.loc[9006,'county']

In [ ]:
gdf.loc[9006,'geometry']

...or by row and column number.  Note that we start counting at zero.

In [ ]:
gdf.iloc[9006,2]

In [ ]:
gdf.iloc[9006,858]

### Retrieving Data from Columns.

In [ ]:
gdf['county']

In [ ]:
gdf['total_pop']

### Retrieving Data from Rows.

In [ ]:
gdf.loc[3]

In [ ]:
gdf.loc[8]

### Creating SubDataFrames

We can create a smaller dataframe from particular rows and columns...

In [ ]:
gdf.loc[[2,6],['county','total_pop']]

Or we can include a range of rows and columns.

In [ ]:
gdf.loc[2:6, 'county':'total_pop']

We can even filter the dataframe.

In [ ]:
gdf[gdf['total_pop'] >= 20000]

In [ ]:
gdf[gdf['county'].isin(['Dallas', 'Denton', 'Tarrant'])]

### Plotting GeoDataFrames

The geometries of GeoDataFrames can be plotted.

In [ ]:
gdf.plot()

In [ ]:
gdf.plot(column = 'county')

In [ ]:
gdf.plot(column = 'cd')

Filtered GeoDataFrames too!

In [ ]:
gdf[gdf['county'].isin(['Dallas', 'Denton', 'Tarrant'])].plot(column = 'county')

### Saving a GeoDataFrame

You can save your GeoDataFrame as a GeoJSON file.  This will take some time for a large dataframe like ours.  **Note that Colab's file folders do not provide storage after the runtime ends.  You must either store a file on Google Drive or download it to your local machine to keep it.**  

In [ ]:
# This takes 30+ minutes for our dataframe.
%%script false  <--- This line prevents this cell from running.  Remove to use cell.

filepath = ""  #Write destination filepath string here.  End with .geojson
gdf.to_file(filepath, driver='GeoJSON')

---
## 3 - Run GerryChain on Graph

Gerrychain creates a large set of partitions on a graph. Texas will be represented as a graph, and each partition represents a possible way to break Texas into single member voting districts.

First we import various commands that Gerrychain uses.

In [ ]:
from gerrychain import Graph, Partition, Election, MarkovChain, GeographicPartition
from gerrychain.updaters import Tally, cut_edges
from gerrychain.constraints import single_flip_contiguous, contiguous
from gerrychain import constraints
from gerrychain.proposals import propose_random_flip, recom
from gerrychain.accept import always_accept
from functools import partial

### a - Creating a graph of our geometries using GerryChain.  

Here we are converting Texas into a graph. The vertex set of this graph will be our Voting Tabulation Districts, or VTDs. Edges will connect VTDs when they share a boundary.

In [ ]:
#  You will get some error messages here.  All is well.  We will talk about this.
graph = Graph.from_geodataframe(gdf)

### Optional - Prepping the graph for GerryChain

Some graphs may not satisfy the constraints you wish to use in GerryChain.  For instance, a graph may not be connected and so cause a violation of GerryChains "contiguous" constraint for its districts.  One can interact and edit the graph using the networkx package until it is ready for use in GerryChain.  

In [ ]:
import networkx as nx
nx.is_connected(graph)


### b - Create an Initial Partition for Gerrychain

Gerrychain generates the set of partitions by taking a random walk through the statespace of valid partitions. This step creates the seed for the random walk.  The seed we are currently using is the U.S. Congressional District map of Texas (117th Congress), which has 36 single-member districts as determined by apportionment following the 2010 census.  Texas will soon have 38 districts, according to 2020 census apportionment.

In [ ]:
initial_partition = Partition(
    graph,
    assignment="cd",
    updaters={
        "cut_edges": cut_edges,
        "population": Tally("total_pop", alias="population")
    }
)


### c - Use Gerrychain to Generate a Collection of District Plans.

We start by defining some quantities that GerryChain will use to create its collection of partitions.

In [ ]:
# Tallies the total population of the graph for later use.
total_pop = 0
for node in graph.nodes:
  total_pop+=graph.nodes[node]['total_pop']

# Creates a list of district names for later use.
districts = []
for node in graph.nodes:
  vtd_district = graph.nodes[node]['cd']
  if vtd_district not in districts:
    districts.append(vtd_district)
num_districts = len(districts)

# Defines the target population per district.
pop_target = int(total_pop/ num_districts)

# Defines the allowable deviation from the target population.
pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, 0.35, pop_key = 'population' )

# Tells GerryChain how to change a partition to create a new partition.
proposal = partial(recom, pop_col='total_pop', pop_target=pop_target, epsilon=.1, node_repeats=10)

We now ask GerryChain to create the collection of partitions.

In [ ]:
from IPython.display import clear_output

total_steps = 100   # Write the total number of partitions you wish to generate.

chain = MarkovChain(
    proposal=proposal, # Tells GerryChain how to create a new partition.
    constraints=[contiguous,pop_constraint],  #  New partitions that do not meet the constraints are discarded.
    accept=always_accept,
    initial_state=initial_partition,
    total_steps=total_steps
)
partition_counter = 0
for partition in chain:  # This loop writes the each VTD's assigned district for each partition in the chain.
    for vtd in gdf.index:
      gdf.loc[vtd,'cd_'+str(partition_counter)] = partition.assignment[vtd]
    clear_output()
    print('Partition '+str(partition_counter) + ' of '+ str(total_steps-1) + ' recorded to GeoDataFrame.')
    partition_counter+=1

See the partitions recorded in the final columns of the table?

In [ ]:
gdf.head(3)

In the following two plots, compare our initial partition (U.S. Congressional Districts) to the partition 100 steps down the chain.

In [ ]:
gdf.plot(column="cd", figsize = (12,12), cmap = 'tab20')

In [ ]:
gdf.plot(column="cd_99", figsize = (12,12), cmap = 'tab20')

---
## 4 - Extract and Use Data from Partitions Recorded on the GeoDataFrame

Now it's time to run an experiment.  There are a few examples below that you are welcome to use or edit.  For your reference, here are the available columns in the GeoDataFrame.

In [ ]:
for col in gdf.columns:
  print(col)

### Example A - Polsby-Popper

We can calculate the Polsby-Popper value over all districts in a partition.  Note that for any given polygon...

In [ ]:
gdf.loc[0,'geometry']

...we can calculate its area...

In [ ]:
gdf.loc[0,'geometry'].area

... and its perimeter.

In [ ]:
gdf.loc[0,'geometry'].length

This means we can calculate its Polsby-Popper score: $\frac{4\pi\cdot A}{P^2}$.  This score, which falls between 0 and 1, is often used as a measure of compactness.

In [ ]:
from math import pi

4*pi*gdf.loc[0,'geometry'].area/(gdf.loc[0,'geometry'].length)**2

In [ ]:
from math import pi
from shapely.ops import unary_union

def PolsbyPopper(partition_col):
  #  We start by making a dictionary of lists of polygons by district.
  polygon_list_dict = {district: [gdf.loc[vtd,'geometry'] for vtd in gdf[gdf[partition_col] == district].index] 
                       for district in districts}
  
  #  We then find the union of each list of polygons to make a single polygon 
  #  for the entire district.
  district_poly_dict = {district:unary_union(polygon_list_dict[district]) for district in districts}
  
  #  We find the Polsby Popper Score of each district polygon.
  PP_dict = {district: 4*pi*district_poly_dict[district].area/(district_poly_dict[district].length)**2 for district in districts}
  return PP_dict

Let's calculate the Polsby-Popper score for every district in our original partition.

In [ ]:
PolsbyPopper('cd')

Now lets calculate the Polsby-Popper scores in each partition.

In [ ]:
# This calculation will take ~18 minutes.

PP_in_partitions = []
for i in range(0,100):
  clear_output()
  print('Calculating Polsby Popper for Partition ' + str(i)+'.')
  PP_in_partitions.append(list(PolsbyPopper('cd_'+str(i)).values()))

Let's plot our data!

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

plt.figure(figsize=(20,8))
plt.xlabel('Partition Number')
plt.ylabel('Polsby Popper')
plt.boxplot(PP_in_partitions)
plt.show()

### Example B - Correlation of Two Groups Across Districts

Choose a pair of columns to compare. If the difference between the number of column 1 citizens and column 2 citizens is less than 10% of their average, then we will declare those groups to be similar in that district.  

First we will choose two columns to compare. Then we will count the number of districts which are similar in each of our partitions.  In the end, we will have a distribution across all partitions: how often do these groups have similar representations?

In [ ]:
col_1 = 'hisp'  # Choose a pair of columns to compare!
col_2 = 'nonhisp'

In [ ]:
#  We create a command that sums the given column's population in each district.
def column_pop_by_district(col, partition_col):
  district_pop = {district:0 for district in districts}
  for vtd in gdf.index:
    vtd_district = gdf.loc[vtd,partition_col]
    district_pop[vtd_district] += gdf.loc[vtd][col]
  return district_pop

#  We create a command that compares the difference between the columns' district 
#  populations against 10% of their average population.
def is_correlated(col_1, col_2, partition_col):
  col_1_pop = column_pop_by_district(col_1,partition_col)
  col_2_pop = column_pop_by_district(col_2,partition_col)
  return {district: abs(col_1_pop[district] - col_2_pop[district]) < \
          .1*(col_1_pop[district] + col_2_pop[district])/2 \
          for district in districts}

#  We count the coorrelated districts.
def num_correlated_districts(col_1, col_2, partition_col):
  return list(is_correlated(col_1,col_2,partition_col).values()).count(True)


Now we count the number of correlated districts in each partition. 

In [ ]:
#  This will take ~24 minutes.
num_corr_district_list = []
for i in range(0,100):
  clear_output()
  print('Computing correlated districts for Partition ' + str(i) + '.')
  num_corr_district_list.append(num_correlated_districts(col_1,col_2,'cd_'+str(i)))

Now let's plot our data. You can interpret your graph in terms of the degree in which your two groups are situated regionally in similar ways.

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(12,8))
plt.xlabel('Partition Number')
plt.ylabel('Correlated Districts')
plt.scatter(range(100),num_corr_district_list, s = 15, color='orangered')
plt.show()

### Example C - Efficiency Gap

Using the data found in two columns of the GeoDataFrame, the following cells calculate the efficiency gap favoring column 1.  The efficiency gap is defined to be the difference between two groups wasted votes, divided by the total number of votes from the two groups.  In each district, every vote by the losing party is wasted.  The winning party's only wasted votes are those beyond what is necessary to win the district.

In [ ]:
# Choose a pair of columns to compare!
col_1 = 'President_2020_D_Biden_general'
col_2 = 'President_2020_R_Trump_general'

Note that this experiment does not provide information about a presidential election, because votes are aggregated statewide, and not by district.

However, we may consider presidential votes to be a proxy for the vote for a local representative. This then gives insight into how votes are wasted in single-member district elections which comprise the state, taken together.

In [ ]:
from math import floor

#  This counts the votes for each column's citizens in each district.
def votes(col_1, col_2, district_col='cd', district_list = districts):
  return {district:
         {col_1: sum([gdf.loc[vdt,col_1] for vdt in gdf[gdf[district_col] == district].index]), 
          col_2: sum([gdf.loc[vdt,col_2] for vdt in gdf[gdf[district_col] == district].index])
          } for district in district_list}

#  We determine the number of votes necessary to have a majority.
def votes_to_win(col_1, col_2, district_col='cd', district_list = districts):
  votes_dict = votes(col_1,col_2,district_col,district_list)
  return {district: floor((votes_dict[district][col_1] + votes_dict[district][col_2])/2)+1 for district in district_list }

#  This calculates the number of wasted votes among each column's citizens.
def wasted_votes(col_1, col_2, district_col = 'cd', district_list = districts):
  votes_dict = votes(col_1, col_2, district_col,district_list)
  to_win_dict = votes_to_win(col_1, col_2, district_col, district_list)
  res = {col_1:0, col_2:0}
  for district in district_list:
    if votes_dict[district][col_1] > votes_dict[district][col_2]:
      res[col_1] += votes_dict[district][col_1] - to_win_dict[district]
      res[col_2] += votes_dict[district][col_2]
    elif votes_dict[district][col_1] < votes_dict[district][col_2]:
      res[col_1] += votes_dict[district][col_1] 
      res[col_2] += votes_dict[district][col_2] - to_win_dict[district]
    else:
      res[col_1] += votes_dict[district][col_1] 
      res[col_2] += votes_dict[district][col_2]
  return res

#  This calculates efficiency gap.
def efficiency_gap(col_1, col_2, district_col = 'cd', district_list = districts):
  total_voters = sum([gdf.loc[vtd,col_1] for vtd in gdf.index]) + \
  sum([gdf.loc[vtd,col_2] for vtd in gdf.index])
  
  wasted_dict = wasted_votes(col_1, col_2,district_col, district_list)

  e_gap = (wasted_dict[col_2] - wasted_dict[col_1])/total_voters

  return e_gap

Here is the efficiency gap if the voting districts had been determined by Partition 9.

In [ ]:
efficiency_gap(col_1, col_2, district_col='cd_9')

Let's make a list of efficiency gaps over all 100 partitions in the GeoDataFrame.

In [ ]:
# This takes 2 minutes.
efficiency_list = []
for i in range(0,100):
  clear_output()
  print('Calculating efficiency gap for Partition ' + str(i) + '.')
  efficiency_list.append(efficiency_gap(col_1, col_2, 'cd_'+str(i)))

Let's plot them!

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(12,8))
plt.xlabel('Partition Number')
plt.ylabel('Efficiency Gap')
plt.bar(range(0,100),efficiency_list, color='blueviolet')
plt.show()

## Part II - Getting and Preparing Data

Please follow [this link](https://colab.research.google.com/drive/1zckE-hLODHXvl6A_NABQjIqneyyVfLCh?usp=sharing#scrollTo=dYR8-ByOESHG) for the second part of our workshop.